In [42]:
# Do pracy z danymi (tabele, ramki danych)
import pandas as pd
# Do budowy modelu i przygotowania danych
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Do zapisu naszego wytrenowanego modelu
import pickle
# Do obliczeń daty
from datetime import datetime

print("Biblioteki zaimportowane pomyślnie!")

Biblioteki zaimportowane pomyślnie!


In [43]:
import pandas as pd
import s3fs # Upewnijmy się, że importujemy s3fs

# --- KONFIGURACJA DOSTĘPU DO DIGITALOCEAN SPACES ---
# Wstaw tutaj swoje dane!
DO_SPACES_ACCESS_KEY = 'DO00WVQ7NF2UKZ29YE84'
DO_SPACES_SECRET_KEY = 'dvpAkDxkLnN2eNlDSpM9Gso+sCxf9qx0/c+bP+QYJr8'
DO_SPACES_ENDPOINT_URL = 'https://fra1.digitaloceanspaces.com'
DO_SPACES_BUCKET_NAME = 'pracadomowa9'

# --- BARDZIEJ NIEZAWODNA METODA POŁĄCZENIA ---
# Tworzymy słownik z opcjami połączenia, który podamy bezpośrednio do pandas
storage_options = {
    'key': DO_SPACES_ACCESS_KEY,
    'secret': DO_SPACES_SECRET_KEY,
    'client_kwargs': {
        'endpoint_url': DO_SPACES_ENDPOINT_URL
    }
}


# --- WCZYTYWANIE DANYCH ---
try:
    # Tworzymy ścieżki do plików w chmurze
    path_2023 = f's3://{DO_SPACES_BUCKET_NAME}/halfmarathon_wroclaw_2023__final.csv'
    path_2024 = f's3://{DO_SPACES_BUCKET_NAME}/halfmarathon_wroclaw_2024__final.csv'

    # Podajemy `storage_options` bezpośrednio do funkcji, aby uniknąć problemów
    df_2023 = pd.read_csv(path_2023, storage_options=storage_options, sep=';')
    df_2024 = pd.read_csv(path_2024, storage_options=storage_options, sep=';')

    # Połącz obie ramki danych w jedną
    df_raw = pd.concat([df_2023, df_2024], ignore_index=True)

    print("Dane zostały pomyślnie wczytane i połączone z DigitalOcean Spaces!")
    print(f"Całkowita liczba wierszy: {len(df_raw)}")
    display(df_raw.head())

except Exception as e:
    print(f"BŁĄD: Mimo udanego połączenia, wystąpił problem z wczytaniem pliku CSV.")
    print(e)

Dane zostały pomyślnie wczytane i połączone z DigitalOcean Spaces!
Całkowita liczba wierszy: 21957


,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
0,1.0,1787,TOMASZ,GRYCKO,NaN,POL,UKS BLIZA WŁADYSŁAWOWO,M,1.0,M30,...,2.926667,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,WROCŁAW,POL,ARKADIUSZGARDZIELEWSKI.PL,M,2.0,M30,...,2.983333,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875
2,3.0,3832,KRZYSZTOF,HADAS,POZNAŃ,POL,NaN,M,3.0,M20,...,3.123333,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475
3,4.0,416,DAMIAN,DYDUCH,KĘPNO,POL,AZS POLITECHNIKA OPOLSKA,M,4.0,M30,...,3.196667,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963
4,5.0,8476,KAMIL,MAŃKOWSKI,MIRKÓW,POL,PARKRUN WROCŁAW,M,5.0,M20,...,3.276667,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654


In [44]:
# --- KROK 1: Wybór potrzebnych kolumn z Twoich plików ---
# Bierzemy '5 km Czas' zamiast '5 km Tempo'
required_columns = ['Rocznik', 'Płeć', '5 km Czas', 'Czas'] 
df_selected = df_raw[required_columns].copy()


# --- KROK 2: Definicja funkcji pomocniczej ---
# Potrzebujemy już tylko jednej funkcji - do konwersji czasu na sekundy.
def time_to_seconds(t):
    try:
        # Sprawdzamy, czy t jest tekstem, zanim użyjemy .split()
        if isinstance(t, str):
            parts = list(map(int, t.split(':')))
            if len(parts) == 3: # Format HH:MM:SS
                h, m, s = parts
                return h * 3600 + m * 60 + s
            elif len(parts) == 2: # Format MM:SS
                m, s = parts
                return m * 60 + s
    except (ValueError, AttributeError):
        return None
    return None # Zwracamy None, jeśli format jest nieprawidłowy


# --- KROK 3: Przetwarzanie danych ---

# Konwertujemy czas końcowy (półmaraton) na sekundy
df_selected['Final_Time_Seconds'] = df_selected['Czas'].apply(time_to_seconds)

# Konwertujemy czas na 5 km na sekundy
df_selected['5km_in_seconds'] = df_selected['5 km Czas'].apply(time_to_seconds)

# Obliczamy wiek na podstawie rocznika
current_year = datetime.now().year
df_selected['Age'] = current_year - df_selected['Rocznik']

# --- NAJWAŻNIEJSZA ZMIANA: OBLICZAMY TEMPO ---
# Dzielimy czas na 5km (w minutach) przez dystans (5 km), aby uzyskać tempo w min/km
df_selected['Pace_5k'] = (df_selected['5km_in_seconds'] / 60) / 5

# Zmieniamy nazwę kolumny 'Płeć' na 'Gender' dla spójności
df_selected = df_selected.rename(columns={'Płeć': 'Gender'})


# --- KROK 4: Ostateczne czyszczenie ---
# Wybieramy tylko te kolumny, które będą użyte w modelu i usuwamy wiersze z brakującymi danymi
final_features = ['Age', 'Gender', 'Pace_5k', 'Final_Time_Seconds']
df_cleaned = df_selected[final_features].dropna()
df_cleaned = df_cleaned[df_cleaned['Gender'].isin(['K', 'M'])]


print("Dane zostały wyczyszczone i przygotowane.")
print(f"Liczba wierszy po czyszczeniu: {len(df_cleaned)}")
print("\nPróbka przygotowanych danych:")
display(df_cleaned.head())

Dane zostały wyczyszczone i przygotowane.
Liczba wierszy po czyszczeniu: 17927

Próbka przygotowanych danych:


,Age,Gender,Pace_5k,Final_Time_Seconds
0,33.0,M,2.923333,3899.0
1,39.0,M,2.960000,3983.0
2,29.0,M,3.153333,4104.0
3,37.0,M,3.236667,4216.0
4,30.0,M,3.240000,4227.0


In [45]:
# X to nasze cechy (dane wejściowe dla modelu)
X = df_cleaned[['Age', 'Gender', 'Pace_5k']]
# y to nasz cel (to, co model ma przewidzieć)
y = df_cleaned['Final_Time_Seconds']

# Dzielimy dane w proporcji 80% na trening, 20% na testy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Zbiór treningowy: {len(X_train)} wierszy")
print(f"Zbiór testowy:    {len(X_test)} wierszy")

Zbiór treningowy: 14341 wierszy
Zbiór testowy:    3586 wierszy


In [46]:
# Definiujemy, które kolumny są numeryczne, a które kategoryczne
numeric_features = ['Age', 'Pace_5k']
categorical_features = ['Gender']

# Tworzymy transformator, który wie, co zrobić z każdym rodzajem kolumny.
# Kolumny kategoryczne zostaną zakodowane metodą OneHotEncoder.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])

# Łączymy transformator i model w jeden spójny pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Trenujemy nasz model na danych treningowych!
model_pipeline.fit(X_train, y_train)

print("Model został pomyślnie wytrenowany!")

Model został pomyślnie wytrenowany!


In [47]:
# Sprawdzamy wynik modelu na zbiorze testowym
# Wynik R^2 mówi nam, jaki procent zmienności danych model potrafi wyjaśnić.
# Bliżej 1.0 oznacza lepszy model.
score = model_pipeline.score(X_test, y_test)

print(f"Wynik R^2 modelu na danych testowych: {score:.4f}")

Wynik R^2 modelu na danych testowych: 0.8813


In [48]:
# Nazwa pliku, w którym zapiszemy model
filename = 'marathon_model.pkl'

# Używamy biblioteki pickle do zapisania obiektu modelu
with open(filename, 'wb') as file:
    pickle.dump(model_pipeline, file)

print(f"Model został zapisany w pliku: '{filename}'")

Model został zapisany w pliku: 'marathon_model.pkl'


In [49]:
# --- ZAPISYWANIE MODELU DO DIGITALOCEAN SPACES ---

# Nazwa pliku, który zapisaliśmy lokalnie w poprzednim kroku
local_filename = 'marathon_model.pkl'
# Nazwa, pod jaką plik będzie widoczny w chmurze
cloud_filename = 'marathon_model.pkl'

# Tworzymy klienta s3
session = boto3.session.Session()
s3_client = session.client('s3',
                           endpoint_url=DO_SPACES_ENDPOINT_URL,
                           aws_access_key_id=DO_SPACES_ACCESS_KEY,
                           aws_secret_access_key=DO_SPACES_SECRET_KEY)

# Wgrywamy plik
try:
    s3_client.upload_file(local_filename, DO_SPACES_BUCKET_NAME, cloud_filename)
    print(f"Model '{local_filename}' został pomyślnie wgrany do Space'a jako '{cloud_filename}'.")
except Exception as e:
    print(f"BŁĄD: Nie udało się wgrać modelu do chmury.")
    print(e)

Model 'marathon_model.pkl' został pomyślnie wgrany do Space'a jako 'marathon_model.pkl'.
